In [118]:
import pandas as pd
from datasets import load_dataset
import seaborn as sns
import matplotlib as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [88]:
df = load_dataset("David-Egea/Creditcard-fraud-detection")
df = df['train'].to_pandas()
type(df)

pandas.core.frame.DataFrame

In [89]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [90]:
df["Class"].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  int64  
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [92]:
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [93]:
legit = df[df.Class == 0]
fraud = df[df.Class == 1]

In [94]:
print(legit)
print(fraud)

          Time         V1         V2  ...       V28  Amount  Class
0            0  -1.359807  -0.072781  ... -0.021053  149.62      0
1            0   1.191857   0.266151  ...  0.014724    2.69      0
2            1  -1.358354  -1.340163  ... -0.059752  378.66      0
3            1  -0.966272  -0.185226  ...  0.061458  123.50      0
4            2  -1.158233   0.877737  ...  0.215153   69.99      0
...        ...        ...        ...  ...       ...     ...    ...
284802  172786 -11.881118  10.071785  ...  0.823731    0.77      0
284803  172787  -0.732789  -0.055080  ... -0.053527   24.79      0
284804  172788   1.919565  -0.301254  ... -0.026561   67.88      0
284805  172788  -0.240440   0.530483  ...  0.104533   10.00      0
284806  172792  -0.533413  -0.189733  ...  0.013649  217.00      0

[284315 rows x 31 columns]
          Time        V1        V2        V3  ...       V27       V28  Amount  Class
541        406 -2.312227  1.951992 -1.609851  ...  0.261145 -0.143276    0.00      

In [95]:
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [96]:
total_fraud_samples= fraud.shape[0]
print(total_fraud_samples)

492


In [97]:
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [98]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

Total no.of fraud transactions are: 492

In [99]:
legit_sample = legit.sample(n=total_fraud_samples)
print(legit_sample.shape)

(492, 31)


In [100]:
new_df = pd.concat([legit_sample, fraud], axis=0)

In [101]:
new_df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
222932,143182,1.987370,-0.410268,-0.601078,0.137262,-0.256421,0.225007,-0.678090,0.227608,1.160683,0.024487,0.332275,0.599032,-0.698716,0.278847,0.115861,0.583887,-0.933989,0.580481,0.400906,-0.228583,-0.174545,-0.456090,0.352714,0.211880,-0.406330,-0.645135,0.021020,-0.040499,4.60,0
125414,77664,-2.885759,1.961258,0.641113,-2.110796,-0.478742,-0.009522,-0.569561,-1.860373,0.723642,-0.683012,-1.871819,-0.003244,-0.602576,-0.107880,-1.140623,0.754832,-0.389729,-0.615143,-0.673504,-0.870046,1.996012,-1.796263,0.074918,-0.455142,0.335193,0.887291,-1.566145,-0.809923,7.83,0
45038,42181,-1.542045,-0.428519,1.933986,-0.709562,-0.398871,-0.648172,0.116459,0.141950,1.008929,-0.662764,-0.903066,0.579763,0.334273,-0.917169,-0.969136,0.117203,-0.212626,-0.627019,-0.302019,0.284509,-0.236481,-0.372544,0.192936,0.455488,0.128572,0.811491,0.295169,0.243501,117.00,0
215209,139975,-0.306139,0.393265,0.458180,-2.074616,1.260777,-0.375036,1.076943,-0.226760,-0.096903,-0.692698,-0.004708,-0.085283,-0.716035,0.364560,-0.285569,0.226575,-0.986165,0.113012,0.995200,0.015214,-0.288949,-0.763818,-0.253557,-1.152125,-0.207175,0.567844,-0.164273,-0.132270,4.00,0
275395,166510,-0.313945,0.955770,-0.279123,-0.757508,0.265872,-0.603402,0.477771,0.462283,-0.264890,-0.434669,-0.000030,0.209431,-1.092677,0.828468,-0.979990,0.419512,-0.597757,0.085219,0.386867,-0.184456,-0.208005,-0.653371,0.080192,-0.532990,-0.468833,0.156533,0.107979,0.017353,8.99,0


In [102]:
new_df.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
279863,169142,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,-5.587794,2.115795,-5.417424,-1.235123,-6.665177,0.401701,-2.897825,-4.570529,-1.315147,0.391167,1.252967,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,-3.232153,2.858466,-3.096915,-0.792532,-5.210141,-0.613803,-2.155297,-3.267116,-0.688505,0.737657,0.226138,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,-3.463891,1.794969,-2.775022,-0.418950,-4.057162,-0.712616,-1.603015,-5.035326,-0.507000,0.266272,0.247968,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,-5.245984,1.933520,-5.030465,-1.127455,-6.416628,0.141237,-2.549498,-4.614717,-1.478138,-0.035480,0.306271,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1
281674,170348,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,0.577829,-0.888722,0.491140,0.728903,0.380428,-1.948883,-0.832498,0.519436,0.903562,1.197315,0.593509,-0.017652,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53,1


In [103]:
new_df["Class"].value_counts()

Class
0    492
1    492
Name: count, dtype: int64

In [104]:
X = new_df.drop(columns=["Class"], axis=1)
y = new_df["Class"]

In [105]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.8, random_state=42)

In [107]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [108]:
X_train_pred = model.predict(X_train)
train_data_accuracy = accuracy_score(X_train_pred, y_train)

In [109]:
print(train_data_accuracy)

0.9846938775510204


In [122]:
X_test_pred = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_pred, y_test)

In [121]:
print(test_data_accuracy)

0.9213197969543148


In [119]:
score = classification_report(y_test, X_test_pred)
print(score)

              precision    recall  f1-score   support

           0       0.89      0.97      0.92       393
           1       0.96      0.88      0.92       395

    accuracy                           0.92       788
   macro avg       0.92      0.92      0.92       788
weighted avg       0.92      0.92      0.92       788

